In [1]:
import os, shutil
import numpy as np

from matplotlib import image, pyplot
from skimage.transform import resize


from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout

from keras import callbacks
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, f1_score, recall_score,classification_report,roc_curve, auc 
from sklearn.utils import class_weight

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from PIL import Image,ImageFile
from tensorflow.keras import models
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import optimizers

In [2]:
resultPath = 'C:/Users/HP/Downloads/Results/CNN'

In [3]:
DATA_PATH = "C:/Users/HP/Downloads/Image_Hunter_dataset/"
spamData=[]
label=[]
hashList=[]
count=-1
for folder in os.listdir(DATA_PATH):
    print(">>>Reading ",folder)
    count+=1
    
    for file in os.listdir(DATA_PATH+folder):
        if(str(file).endswith('.jpg') or str(file).endswith('.JPG') or str(file).endswith('.jpeg') or str(file).endswith('.JPEG')):
            img = image.imread(DATA_PATH+folder+'/'+file)
            hsh = hash(tuple(np.array(img).flatten()))
            if(hsh not in hashList):
                spamData.append(resize(img, (156, 156, 3)))
                hashList.append(hsh)
                label.append(count)
spamData=np.array(spamData)
label=np.array(label)

>>>Reading  NaturalImages
>>>Reading  SpamImages


In [4]:
print("Number of SPAM",len(label[label==0]))
print("Number of HAM",len(label[label==1]))

Number of SPAM 810
Number of HAM 895


In [5]:
print("Spam data shape : ",spamData.shape," Label shape : ",label.shape)

Spam data shape :  (1705, 156, 156, 3)  Label shape :  (1705,)


In [6]:
test_valid_Percentage = 0.3
x_train,x_test,y_train,y_test = train_test_split(spamData,label,test_size = test_valid_Percentage,random_state=42, stratify=label,shuffle=True)

In [7]:
print("x_train shape : ",x_train.shape," y_train shape : ",y_train.shape)
print("x_test shape : ",x_test.shape," y_test shape : ",y_test.shape)

x_train shape :  (1193, 156, 156, 3)  y_train shape :  (1193,)
x_test shape :  (512, 156, 156, 3)  y_test shape :  (512,)


In [8]:
print("Number of train SPAM",len(y_train[y_train==0]))
print("Number of train HAM",len(y_train[y_train==1]))

print("Number of test SPAM",len(y_test[y_test==0]))
print("Number of test HAM",len(y_test[y_test==1]))

Number of train SPAM 567
Number of train HAM 626
Number of test SPAM 243
Number of test HAM 269


In [9]:
y_train_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)

print("y_train_oh shape : ",y_train_oh.shape," y_test_oh shape : ",y_test_oh.shape)

y_train_oh shape :  (1193, 2)  y_test_oh shape :  (512, 2)


In [49]:
#Initializing Exception Network as Base Model for Transfer Learning

#from tensorflow.keras.applications.DenseNet201 import DenseNet201
from tensorflow.keras.applications import DenseNet201


base_model_dense = DenseNet201(input_shape=(156, 156,3), weights='imagenet', include_top=False,classes= 2)

for layer in base_model_dense.layers[:699]:
    layer.trainable = False
for layer in base_model_dense.layers[700:]:
    layer.trainable = True

for layer in base_model_dense.layers:
    print(layer.name , layer.trainable)
len(base_model_dense.layers)

input_14 False
zero_padding2d_4 False
conv1/conv False
conv1/bn False
conv1/relu False
zero_padding2d_5 False
pool1 False
conv2_block1_0_bn False
conv2_block1_0_relu False
conv2_block1_1_conv False
conv2_block1_1_bn False
conv2_block1_1_relu False
conv2_block1_2_conv False
conv2_block1_concat False
conv2_block2_0_bn False
conv2_block2_0_relu False
conv2_block2_1_conv False
conv2_block2_1_bn False
conv2_block2_1_relu False
conv2_block2_2_conv False
conv2_block2_concat False
conv2_block3_0_bn False
conv2_block3_0_relu False
conv2_block3_1_conv False
conv2_block3_1_bn False
conv2_block3_1_relu False
conv2_block3_2_conv False
conv2_block3_concat False
conv2_block4_0_bn False
conv2_block4_0_relu False
conv2_block4_1_conv False
conv2_block4_1_bn False
conv2_block4_1_relu False
conv2_block4_2_conv False
conv2_block4_concat False
conv2_block5_0_bn False
conv2_block5_0_relu False
conv2_block5_1_conv False
conv2_block5_1_bn False
conv2_block5_1_relu False
conv2_block5_2_conv False
conv2_block5_c

707

In [50]:
NO_OF_EPOCHS=10
BATCH_SIZE=128

model=Sequential()
model.add(base_model_dense)
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 4, 4, 1920)        18321984  
                                                                 
 flatten_4 (Flatten)         (None, 30720)             0         
                                                                 
 dense_8 (Dense)             (None, 128)               3932288   
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                                                 
Total params: 22,254,401
Trainable params: 4,215,041
Non-trainable params: 18,039,360
_________________________________________________________________


In [51]:
checkpointer = callbacks.ModelCheckpoint(filepath=resultPath+"/checkpoint-{epoch:04d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger(resultPath+'/result_logger.csv',separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.2, patience=2, min_lr=0.001)
model.fit(x_train,y_train,epochs= 10,verbose=1,batch_size= 64,validation_data=(x_test,y_test),callbacks=[checkpointer,csv_logger,reduce_lr])

Epoch 1/10
19/19 [==============================] - 94s 5s/step - loss: 0.5524 - accuracy: 0.9095 - val_loss: 0.1719 - val_accuracy: 0.9883 - lr: 0.0010
Epoch 2/10
19/19 [==============================] - 84s 4s/step - loss: 0.0438 - accuracy: 0.9874 - val_loss: 0.0919 - val_accuracy: 0.9863 - lr: 0.0010
Epoch 3/10
19/19 [==============================] - 84s 4s/step - loss: 0.0187 - accuracy: 0.9925 - val_loss: 0.0527 - val_accuracy: 0.9824 - lr: 0.0010
Epoch 4/10
19/19 [==============================] - 84s 4s/step - loss: 0.0116 - accuracy: 0.9958 - val_loss: 0.0529 - val_accuracy: 0.9883 - lr: 0.0010
Epoch 5/10
19/19 [==============================] - 84s 4s/step - loss: 0.0059 - accuracy: 0.9992 - val_loss: 0.0505 - val_accuracy: 0.9902 - lr: 0.0010
Epoch 6/10
19/19 [==============================] - 84s 4s/step - loss: 0.0033 - accuracy: 0.9992 - val_loss: 0.0565 - val_accuracy: 0.9863 - lr: 0.0010
Epoch 7/10
19/19 [==============================] - 84s 4s/step - loss: 5.4288e-04

In [52]:
prediction_prob1 = model.predict(x_test,verbose=1)
y_pred=np.round(prediction_prob1)

16/16 [==============================] - 28s 2s/step


In [53]:
print(accuracy_score(y_test,y_pred))
print(precision_score(y_test,y_pred,average='binary'))
print(recall_score(y_test,y_pred,average='binary'))
print(f1_score(y_test,y_pred,average='binary'))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.98828125
0.9925093632958801
0.9851301115241635
0.9888059701492536
[[241   2]
 [  4 265]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       243
           1       0.99      0.99      0.99       269

    accuracy                           0.99       512
   macro avg       0.99      0.99      0.99       512
weighted avg       0.99      0.99      0.99       512

